In [1]:
import re
import warnings
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib as mpl # Import base matplotlib

In [2]:
def parse_parameters(parameter_str):
    # Extract the main minimizer and its parameters
    minimizer_match = re.match(r"(\w+)\((.*)\)", parameter_str)
    if not minimizer_match:
        return {}

    minimizer_name = minimizer_match.group(1)
    parameters_str = minimizer_match.group(2)

    # Extract individual parameters using regular expressions
    params = {}
    param_pattern = re.compile(r'(\w+)=([^,]+(?:\([^)]*\))?)')
    for match in param_pattern.finditer(parameters_str):
        key = match.group(1).strip()
        value = match.group(2).strip()
        # Check if value should be converted to float, int, or kept as is
        if re.match(r'^-?\d+(\.\d+)?$', value):
            if '.' in value:
                value = float(value)
            else:
                value = int(value)
        elif value.lower() == 'true':
            value = True
        elif value.lower() == 'false':
            value = False
        params[key] = value

    # Include the minimizer name
    params['minimizer_name'] = minimizer_name
    return params

In [3]:
# --- Load EASE Data ---
# The user must provide the full path to the 'user_results.csv' file
# from the EASE model results.
results_ease = 'YOUR_FILE_PATH_HERE'  # <-- EDIT THIS LINE
# --- Load ItemKNN Data ---
# The user must provide the full path to the 'user_results.csv' file
# from the ItemKNN model results.
results_itemknn = 'YOUR_FILE_PATH_HERE' # <-- EDIT THIS LINE

In [4]:
results_ease = pd.read_csv(results_ease)
results_itemknn = pd.read_csv(results_itemknn)

In [5]:
results_ease.columns

Index(['Unnamed: 0', 'minimizer_identifier', 'estimator_identifier',
       'batch_id', 'per_user_scores', 'per_user_original_input_interactions',
       'per_user_minimized_input_interactions', 'per_user_target_interactions',
       'per_user_minimization_ratios', 'per_user_runtimes',
       'per_user_sample_counts', 'per_user_performance_threshold',
       'per_user_constraint_satisfaction', 'NDCGK_100_test_full_input',
       'NDCGK_100_test_min_input', 'RecallK_20_test_full_input',
       'RecallK_20_test_min_input', 'RecallK_50_test_full_input',
       'RecallK_50_test_min_input', 'CalibratedRecallK_20_test_full_input',
       'CalibratedRecallK_20_test_min_input',
       'CalibratedRecallK_50_test_full_input',
       'CalibratedRecallK_50_test_min_input'],
      dtype='object')

In [6]:
parameters_df = results_ease['minimizer_identifier'].apply(parse_parameters)
parameters_df = pd.json_normalize(parameters_df)
ease_netflix = pd.concat([results_ease, parameters_df], axis=1)

In [7]:
parameters_df = results_itemknn['minimizer_identifier'].apply(parse_parameters)
parameters_df = pd.json_normalize(parameters_df)
itemknn_netflix = pd.concat([results_itemknn, parameters_df], axis=1)

In [24]:
# Define the conditions
condition = ease_netflix['eta'] == 0.98

# Select the rows where the combined condition is True
ease_netflix_filtered= ease_netflix[condition]

condition = itemknn_netflix['eta'] == 0.98
itemknn_netflix_filtered = itemknn_netflix[condition]

In [25]:
ease_netflix_filtered

,Unnamed: 0,minimizer_identifier,estimator_identifier,batch_id,per_user_scores,per_user_original_input_interactions,per_user_minimized_input_interactions,per_user_target_interactions,per_user_minimization_ratios,per_user_runtimes,...,max_size,metric,model,density,l2,remove_history,timeout,timeout_manager,minimizer_name,beam_depth
0,0,"GreedyForwardMinimizer(eta=0.98,max_size=None,...","ExponentialDecayEstimator(K=128, gamma=-0.4)",0,0.980200,72,54.0,128,0.750000,1.047043,...,None,NDCG_100,EASE(alpha=0,0.003,2400),True,345600,<minipack.minimizers.base.TimeoutManager objec...,GreedyForwardMinimizer,NaN
1,1,"GreedyForwardMinimizer(eta=0.98,max_size=None,...","ExponentialDecayEstimator(K=128, gamma=-0.4)",0,0.984670,22,18.0,128,0.818182,0.145386,...,None,NDCG_100,EASE(alpha=0,0.003,2400),True,345600,<minipack.minimizers.base.TimeoutManager objec...,GreedyForwardMinimizer,NaN
2,2,"GreedyForwardMinimizer(eta=0.98,max_size=None,...","ExponentialDecayEstimator(K=128, gamma=-0.4)",0,1.000000,9,8.0,128,0.888889,0.042916,...,None,NDCG_100,EASE(alpha=0,0.003,2400),True,345600,<minipack.minimizers.base.TimeoutManager objec...,GreedyForwardMinimizer,NaN
3,3,"GreedyForwardMinimizer(eta=0.98,max_size=None,...","ExponentialDecayEstimator(K=128, gamma=-0.4)",0,0.980552,68,55.0,128,0.808824,0.976702,...,None,NDCG_100,EASE(alpha=0,0.003,2400),True,345600,<minipack.minimizers.base.TimeoutManager objec...,GreedyForwardMinimizer,NaN
4,4,"GreedyForwardMinimizer(eta=0.98,max_size=None,...","ExponentialDecayEstimator(K=128, gamma=-0.4)",0,0.980467,25,18.0,128,0.720000,0.155297,...,None,NDCG_100,EASE(alpha=0,0.003,2400),True,345600,<minipack.minimizers.base.TimeoutManager objec...,GreedyForwardMinimizer,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
745765,745765,"MostPopularSelectionMinimizer(eta=0.98,metric=...","ExponentialDecayEstimator(K=128, gamma=-0.4)",4,0.980443,367,325.0,128,0.885559,0.173050,...,NaN,NDCG_100,EASE(alpha=0,0.003,2400),True,345600,<minipack.minimizers.base.TimeoutManager objec...,MostPopularSelectionMinimizer,NaN
745766,745766,"MostPopularSelectionMinimizer(eta=0.98,metric=...","ExponentialDecayEstimator(K=128, gamma=-0.4)",4,0.980801,372,317.0,128,0.852151,0.166843,...,NaN,NDCG_100,EASE(alpha=0,0.003,2400),True,345600,<minipack.minimizers.base.TimeoutManager objec...,MostPopularSelectionMinimizer,NaN
745767,745767,"MostPopularSelectionMinimizer(eta=0.98,metric=...","ExponentialDecayEstimator(K=128, gamma=-0.4)",4,0.980127,404,308.0,128,0.762376,0.160289,...,NaN,NDCG_100,EASE(alpha=0,0.003,2400),True,345600,<minipack.minimizers.base.TimeoutManager objec...,MostPopularSelectionMinimizer,NaN
745768,745768,"MostPopularSelectionMinimizer(eta=0.98,metric=...","ExponentialDecayEstimator(K=128, gamma=-0.4)",4,0.980585,385,292.0,128,0.758442,0.149227,...,NaN,NDCG_100,EASE(alpha=0,0.003,2400),True,345600,<minipack.minimizers.base.TimeoutManager objec...,MostPopularSelectionMinimizer,NaN


In [26]:
itemknn_netflix_filtered

,Unnamed: 0,minimizer_identifier,estimator_identifier,batch_id,per_user_scores,per_user_original_input_interactions,per_user_minimized_input_interactions,per_user_target_interactions,per_user_minimization_ratios,per_user_runtimes,...,model,normalize_X,normalize_sim,pop_discount,similarity,remove_history,timeout,timeout_manager,minimizer_name,beam_depth
0,0,"GreedyForwardMinimizer(eta=0.98,max_size=None,...","ExponentialDecayEstimator(K=128, gamma=0.8)",4,0.981668,375,43.0,128,0.114667,4.222637,...,ItemKNN(K=50,True,True,None,cosine),True,345600,<minipack.minimizers.base.TimeoutManager objec...,GreedyForwardMinimizer,NaN
1,1,"GreedyForwardMinimizer(eta=0.98,max_size=None,...","ExponentialDecayEstimator(K=128, gamma=0.8)",4,0.980011,404,82.0,128,0.202970,9.752681,...,ItemKNN(K=50,True,True,None,cosine),True,345600,<minipack.minimizers.base.TimeoutManager objec...,GreedyForwardMinimizer,NaN
2,2,"GreedyForwardMinimizer(eta=0.98,max_size=None,...","ExponentialDecayEstimator(K=128, gamma=0.8)",4,0.981886,422,36.0,128,0.085308,4.067533,...,ItemKNN(K=50,True,True,None,cosine),True,345600,<minipack.minimizers.base.TimeoutManager objec...,GreedyForwardMinimizer,NaN
3,3,"GreedyForwardMinimizer(eta=0.98,max_size=None,...","ExponentialDecayEstimator(K=128, gamma=0.8)",4,0.980646,356,32.0,128,0.089888,2.943287,...,ItemKNN(K=50,True,True,None,cosine),True,345600,<minipack.minimizers.base.TimeoutManager objec...,GreedyForwardMinimizer,NaN
4,4,"GreedyForwardMinimizer(eta=0.98,max_size=None,...","ExponentialDecayEstimator(K=128, gamma=0.8)",4,0.980705,357,56.0,128,0.156863,5.388471,...,ItemKNN(K=50,True,True,None,cosine),True,345600,<minipack.minimizers.base.TimeoutManager objec...,GreedyForwardMinimizer,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
821216,821216,"GreedyBackwardMinimizer(eta=0.98,metric=NDCG_1...","ExponentialDecayEstimator(K=128, gamma=0.8)",4,0.981323,367,33.0,128,0.089918,21.322613,...,ItemKNN(K=50,True,True,None,cosine),True,345600,<minipack.minimizers.base.TimeoutManager objec...,GreedyBackwardMinimizer,NaN
821217,821217,"GreedyBackwardMinimizer(eta=0.98,metric=NDCG_1...","ExponentialDecayEstimator(K=128, gamma=0.8)",4,0.980156,372,74.0,128,0.198925,20.624588,...,ItemKNN(K=50,True,True,None,cosine),True,345600,<minipack.minimizers.base.TimeoutManager objec...,GreedyBackwardMinimizer,NaN
821218,821218,"GreedyBackwardMinimizer(eta=0.98,metric=NDCG_1...","ExponentialDecayEstimator(K=128, gamma=0.8)",4,0.982448,404,44.0,128,0.108911,26.654238,...,ItemKNN(K=50,True,True,None,cosine),True,345600,<minipack.minimizers.base.TimeoutManager objec...,GreedyBackwardMinimizer,NaN
821219,821219,"GreedyBackwardMinimizer(eta=0.98,metric=NDCG_1...","ExponentialDecayEstimator(K=128, gamma=0.8)",4,0.981515,385,43.0,128,0.111688,27.610467,...,ItemKNN(K=50,True,True,None,cosine),True,345600,<minipack.minimizers.base.TimeoutManager objec...,GreedyBackwardMinimizer,NaN


In [27]:
columns_to_average = [
    'NDCGK_100_test_min_input',
    'NDCGK_100_test_full_input',
    'CalibratedRecallK_20_test_full_input',
    'CalibratedRecallK_20_test_min_input',
    'CalibratedRecallK_50_test_full_input',
    'CalibratedRecallK_50_test_min_input'
]

In [28]:
ease_netflix_average_metrics_by_minimizer = ease_netflix_filtered.groupby('minimizer_name')[columns_to_average].mean()

In [29]:
ease_netflix_average_metrics_by_minimizer

,NDCGK_100_test_min_input,NDCGK_100_test_full_input,CalibratedRecallK_20_test_full_input,CalibratedRecallK_20_test_min_input,CalibratedRecallK_50_test_full_input,CalibratedRecallK_50_test_min_input
minimizer_name,,,,,,
EmbeddingSimilaritySelectionMinimizer,0.376755,0.382349,0.344818,0.341422,0.432243,0.426426
GreedyBackwardMinimizer,0.375468,0.382349,0.344818,0.335299,0.432243,0.418944
GreedyBeamForwardMinimizer,0.376700,0.382349,0.344818,0.336438,0.432243,0.421656
GreedyForwardMinimizer,0.377166,0.382349,0.344818,0.337048,0.432243,0.423460
LeastPopularSelectionMinimizer,0.380433,0.382349,0.344818,0.342275,0.432243,0.430070
MostPopularSelectionMinimizer,0.374479,0.382349,0.344818,0.338842,0.432243,0.423516
RandomSelectionMinimizer,0.378722,0.382349,0.344818,0.340757,0.432243,0.427636


In [30]:
ease_netflix_average_metrics_by_minimizer["NDCG_ratio"] = ease_netflix_average_metrics_by_minimizer["NDCGK_100_test_min_input"] / ease_netflix_average_metrics_by_minimizer["NDCGK_100_test_full_input"]
ease_netflix_average_metrics_by_minimizer["Recall_20_ratio"] = ease_netflix_average_metrics_by_minimizer["CalibratedRecallK_20_test_min_input"] / ease_netflix_average_metrics_by_minimizer["CalibratedRecallK_20_test_full_input"]
ease_netflix_average_metrics_by_minimizer["Recall_50_ratio"] = ease_netflix_average_metrics_by_minimizer["CalibratedRecallK_50_test_min_input"] / ease_netflix_average_metrics_by_minimizer["CalibratedRecallK_50_test_full_input"]

In [31]:
ease_netflix_average_metrics_by_minimizer

,NDCGK_100_test_min_input,NDCGK_100_test_full_input,CalibratedRecallK_20_test_full_input,CalibratedRecallK_20_test_min_input,CalibratedRecallK_50_test_full_input,CalibratedRecallK_50_test_min_input,NDCG_ratio,Recall_20_ratio,Recall_50_ratio
minimizer_name,,,,,,,,,
EmbeddingSimilaritySelectionMinimizer,0.376755,0.382349,0.344818,0.341422,0.432243,0.426426,0.985368,0.990151,0.986542
GreedyBackwardMinimizer,0.375468,0.382349,0.344818,0.335299,0.432243,0.418944,0.982001,0.972393,0.969230
GreedyBeamForwardMinimizer,0.376700,0.382349,0.344818,0.336438,0.432243,0.421656,0.985225,0.975696,0.975507
GreedyForwardMinimizer,0.377166,0.382349,0.344818,0.337048,0.432243,0.423460,0.986444,0.977466,0.979679
LeastPopularSelectionMinimizer,0.380433,0.382349,0.344818,0.342275,0.432243,0.430070,0.994988,0.992625,0.994972
MostPopularSelectionMinimizer,0.374479,0.382349,0.344818,0.338842,0.432243,0.423516,0.979416,0.982668,0.979808
RandomSelectionMinimizer,0.378722,0.382349,0.344818,0.340757,0.432243,0.427636,0.990512,0.988221,0.989341


In [32]:
itemknn_netflix_average_metrics_by_minimizer = itemknn_netflix_filtered.groupby('minimizer_name')[columns_to_average].mean()

In [33]:
itemknn_netflix_average_metrics_by_minimizer["NDCG_ratio"] = itemknn_netflix_average_metrics_by_minimizer["NDCGK_100_test_min_input"] / itemknn_netflix_average_metrics_by_minimizer["NDCGK_100_test_full_input"]
itemknn_netflix_average_metrics_by_minimizer["Recall_20_ratio"] = itemknn_netflix_average_metrics_by_minimizer["CalibratedRecallK_20_test_min_input"] / itemknn_netflix_average_metrics_by_minimizer["CalibratedRecallK_20_test_full_input"]
itemknn_netflix_average_metrics_by_minimizer["Recall_50_ratio"] = itemknn_netflix_average_metrics_by_minimizer["CalibratedRecallK_50_test_min_input"] / itemknn_netflix_average_metrics_by_minimizer["CalibratedRecallK_50_test_full_input"]

In [34]:
itemknn_netflix_average_metrics_by_minimizer

,NDCGK_100_test_min_input,NDCGK_100_test_full_input,CalibratedRecallK_20_test_full_input,CalibratedRecallK_20_test_min_input,CalibratedRecallK_50_test_full_input,CalibratedRecallK_50_test_min_input,NDCG_ratio,Recall_20_ratio,Recall_50_ratio
minimizer_name,,,,,,,,,
EmbeddingSimilaritySelectionMinimizer,0.320112,0.323678,0.282264,0.280431,0.37545,0.371742,0.988984,0.993505,0.990122
GreedyBackwardMinimizer,0.317206,0.323678,0.282264,0.275296,0.37545,0.367988,0.980005,0.975314,0.980126
GreedyBeamForwardMinimizer,0.319799,0.323678,0.282264,0.278775,0.37545,0.369937,0.988017,0.987639,0.985316
GreedyForwardMinimizer,0.320064,0.323678,0.282264,0.278530,0.37545,0.370790,0.988834,0.986771,0.987588
LeastPopularSelectionMinimizer,0.327076,0.323678,0.282264,0.286394,0.37545,0.379619,1.010501,1.014630,1.011104
MostPopularSelectionMinimizer,0.314424,0.323678,0.282264,0.275370,0.37545,0.366751,0.971410,0.975575,0.976830
RandomSelectionMinimizer,0.320943,0.323678,0.282264,0.279705,0.37545,0.372831,0.991550,0.990932,0.993024
